In [1]:
import os
import sys
import time
import datetime
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
sys.path.append('/Users/yantingting/PycharmProjects/modeling/')
import database_conncet as data_con
import data_processing as dp
import summary_statistics as ss
from data_io_utils import *

In [2]:
date = datetime.date.today().strftime('%m-%d')
file_path= os.path.join('/Users/yantingting/Seafile/风控/模型/10 印尼/低费率测试/ytt/', date)
if not os.path.exists(file_path):
    os.mkdir(file_path)

# 老客户分布

In [3]:
query= '''
SELECT date(apply_time) as apply_date
			, grouping
			, approved_period
			,count(id) as apply_cnt
			,count(case when loan_status not in  ('DENIED','RESCIND') then id end) as dd_cnt
	  FROM dw_gocash_go_cash_loan_gocash_core_loan
	  WHERE apply_time :: timestamp >= '2020-01-17 12:00:00' and return_flag = 'true' 
	  		and customer_type = 'ONE_KEY_LOAN' and grouping in ('OneControlGroup','OneTest1','OneTest2')
	  GROUP BY 1,2,3	
'''

In [4]:
df = data_con.database_gn(query)
save_data_to_pickle(df, file_path, '老客户转化.pkl')
df.shape

Successful connection


(108, 5)

In [5]:
df = load_data_from_pickle(file_path,'老客户转化.pkl')
df.head()

,apply_date,grouping,approved_period,apply_cnt,dd_cnt
0,2020-01-17,OneControlGroup,8,23,23
1,2020-01-17,OneControlGroup,15,351,351
2,2020-01-17,OneTest1,8,20,20
3,2020-01-17,OneTest1,15,401,401
4,2020-01-17,OneTest2,8,26,26


In [12]:
frame1 = pd.pivot_table(df, index = 'apply_date',columns = 'grouping', values=('apply_cnt'), aggfunc='sum')
frame1['all'] = frame1.apply(lambda x: x.sum(), axis = 1)
list1 = ['OneControlGroup',	'OneTest1',	'OneTest2']
for cols in list1:
    frame1[cols + '1'] = frame1[cols]/frame1['all']
frame1

grouping,OneControlGroup,OneTest1,OneTest2,all,OneControlGroup1,OneTest11,OneTest21
apply_date,,,,,,,
2020-01-17,374,421,514,1309,0.285714,0.321620,0.392666
2020-01-18,575,768,772,2115,0.271868,0.363121,0.365012
2020-01-19,524,643,675,1842,0.284473,0.349077,0.366450
2020-01-20,711,900,893,2504,0.283946,0.359425,0.356629
2020-01-21,607,754,819,2180,0.278440,0.345872,0.375688
2020-01-22,587,755,728,2070,0.283575,0.364734,0.351691
2020-01-23,241,636,1082,1959,0.123022,0.324655,0.552323
2020-01-24,132,654,1380,2166,0.060942,0.301939,0.637119
2020-01-25,137,581,1196,1914,0.071578,0.303553,0.624869


# 逾期表现

In [13]:
query2 = '''

	select
		t1.id loan_id,
        t1."grouping",
		effective_date,
		return_flag,
		loan_status,
		approved_period,
		late_date,
		approved_principal,
		media_source,
		case
			when loan_status = 'COLLECTION' then current_date-late_date
			else round(late_fee /(approved_principal*0.025))::int end as DPD,
		case when late_date-effective_date = approved_period then 1 else 0 end as flag1
		from
			(
			select
				*
			from
				dw_gocash_go_cash_loan_gocash_core_loan
			where
				"grouping" != '' and effective_date!='1970-01-01' and loan_status not in ('DENIED',
				'RESCIND',
				'APPROVING',
				'CREATED')) t1
		left join (
			select
				*
			from
				(
				select
					order_id,
					late_date,
					row_number() over(partition by order_id
				order by
					late_date asc) as num
				from
					dw_gocash_gocash_collection_col_case
				where
					(order_status = 'COLLECTION_PAIDOFF'
					or order_status = 'COLLECTION')
					and app_id not in ('Credits',
					'KASANDAAI'))t
			where
				num = 1) t2 on
			t1.id = t2.order_id
			left join (
				select
					id,
					media_source
				from
					public.dw_gocash_go_cash_loan_gocash_core_customer
				) t3 on
			t1.customer_id = t3.id
'''

In [14]:
df2 = data_con.database_gn(query2)
save_data_to_pickle(df2, file_path, '逾期表现.pkl')
df2.shape

Successful connection


(49007, 11)

In [15]:
df2.to_excel(os.path.join(file_path,'temp1.xlsx'))

In [ ]:
8期截止到当前有1+表现的是